首先导入使用的包，使用request进行网页请求，使用pandas保存数据

In [ ]:
import requests
import pandas as pd
import time 

设置请求头，伪装为浏览器

In [4]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.13 Safari/537.36'
}

发起请求，将找到的第一个数据源作为请求目标

In [5]:
url = 'https://c.m.163.com/ug/api/wuhan/app/data/list-total'   # 定义要访问的地址 
r = requests.get(url, headers=headers)  # 使用requests发起请求

In [6]:
print(r.status_code)  # 查看请求状态

200


In [7]:
print(type(r.text))
print(len(r.text))

<class 'str'>
321018


可以看到返回后的内容是一个几十万长度的字符串，由于字符串格式不方便进行分析，并且在网页预览中发现数据为类似字典的json格式，所以我们将其转为json格式

In [8]:
import json
data_json = json.loads(r.text)

In [9]:
type(data_json)

dict

In [10]:
data_json.keys()

dict_keys(['reqId', 'code', 'msg', 'data', 'timestamp'])

可以看出在data中存放着我们需要的数据，因此我们取出数据

In [12]:
data = data_json['data'] # 取出json中的数据
data.keys()

dict_keys(['chinaTotal', 'chinaDayList', 'lastUpdateTime', 'overseaLastUpdateTime', 'areaTree'])

 数据中总共有四个键，每个键存储着不同的内容：


|键名称 |数据内容|
|:--|--:|
|chinaTotal|全国当日数据|
|chinaDayList|全国历史数据|
|lastUpdateTime|更新时间|
|areaTree|世界各地实时数据|

接下来我们分别获取实时数据和历史数据。

### 实时数据爬取
#### 全国各省实时数据爬取

首先我们抓取全国各省的实时数据，在areaTree键值对中，存放着世界各地的实时数据，areaTree是一个列表，每一个元素都是一个国家的数据，每一个元素的children是各国家省份的数据

In [13]:
data_province = data['areaTree'][2]['children']  # 取出中国各省的实时数据children

In [14]:
type(data_province)

list

data_province是列表格式，每个元素是一个省的实时数据，并且为字典格式，每个省的键名称全部相同

In [15]:
data_province[0].keys() # 查看每个省键名称

dict_keys(['today', 'total', 'extData', 'name', 'id', 'lastUpdateTime', 'children'])

各省数据键值含义如下：<br>

|键名称|数据内容|
|:--|--:|
|today|各省当日数据|
|total|各省当日累计数据|
|extData|无任何数据|
|name|各省名称|
|id |各省行政编号|
|lastUpdateTime|更新时间|
|children|各省下一级数据|

In [16]:
for i in range(len(data_province)): # 遍历查看各省名称、更新时间
    print(data_province[i]['name'],data_province[i]['lastUpdateTime'])
    if i == 5:
        break

湖北 2020-07-01 00:04:02
广东 2020-07-01 08:23:59
河南 2020-07-01 00:04:01
浙江 2020-07-01 00:04:02
香港 2020-07-01 10:43:48
湖南 2020-07-01 00:04:02


用DataFrame以字典格式生成数据的例子，传入一个列表，列表每一个元素都是字典

In [17]:
test_dict = [{'a':1,
              'b':2,
              'c':3,},
             
             {'a':111,
              'b':222}]
pd.DataFrame(test_dict)

,a,b,c
0,1,2,3.0
1,111,222,NaN


In [18]:
pd.DataFrame(data_province).head() # 直接提取数据效果并不理想

,today,total,extData,name,id,lastUpdateTime,children
0,"{'confirm': 0, 'suspect': None, 'heal': 0, 'de...","{'confirm': 68135, 'suspect': 0, 'heal': 63623...",{},湖北,420000,2020-07-01 00:04:02,"[{'today': {'confirm': 0, 'suspect': None, 'he..."
1,"{'confirm': 0, 'suspect': None, 'heal': 0, 'de...","{'confirm': 1641, 'suspect': 0, 'heal': 1627, ...",{},广东,440000,2020-07-01 08:23:59,"[{'today': {'confirm': 0, 'suspect': None, 'he..."
2,"{'confirm': 0, 'suspect': None, 'heal': 0, 'de...","{'confirm': 1276, 'suspect': 0, 'heal': 1254, ...",{},河南,410000,2020-07-01 00:04:01,"[{'today': {'confirm': 0, 'suspect': None, 'he..."
3,"{'confirm': 0, 'suspect': None, 'heal': 0, 'de...","{'confirm': 1269, 'suspect': 0, 'heal': 1267, ...",{},浙江,330000,2020-07-01 00:04:02,"[{'today': {'confirm': 0, 'suspect': None, 'he..."
4,"{'confirm': 2, 'suspect': None, 'heal': 2, 'de...","{'confirm': 1205, 'suspect': 0, 'heal': 1107, ...",{},香港,810000,2020-07-01 10:43:48,"[{'today': {'confirm': 2, 'suspect': None, 'he..."


不能直接生成DataFrame是因为数据中嵌套着字典，例如湖北省数据如下：标红线表示带有嵌套字典，篮筐内没有嵌套字典。
<img src="https://imgkr.cn-bj.ufileos.com/5714dd10-2a60-44fc-ab80-33ff1485f673.png">

逐个拆解数据，我们已经了解children为下一地级数据，我们只分析到省单位，因此各省的children数据不采集；extData为空值，也不采集，最后具体采集方法如下：

不需要采集的数据：children、extData。
需要采集的数据：由于数据中today和total嵌套着字典，因此不能直接获取，对于id、lastUpdateTime、name、可以直接取出为一个数据，today为一个数据，total为一个数据，最后三个数据合并为一个数据。

In [19]:
# 获取id、lastUpdateTime、name
info = pd.DataFrame(data_province)[['id','lastUpdateTime','name']]
info.head()

,id,lastUpdateTime,name
0,420000,2020-07-01 00:04:02,湖北
1,440000,2020-07-01 08:23:59,广东
2,410000,2020-07-01 00:04:01,河南
3,330000,2020-07-01 00:04:02,浙江
4,810000,2020-07-01 10:43:48,香港


In [20]:
info.shape

(34, 3)

列表推导式例子

In [21]:
l1 = [1,1,1,2,2,2]

In [22]:
l2 = [i+1 for i in l1 ]
l2

[2, 2, 2, 3, 3, 3]

In [23]:
# 获取today中的数据
today_data = pd.DataFrame([province['today'] for province in data_province ]) 
today_data.head()

,confirm,suspect,heal,dead,severe,storeConfirm
0,0,None,0,0,None,0
1,0,None,0,0,None,0
2,0,None,0,0,None,0
3,0,None,0,0,None,0
4,2,None,2,0,None,0


In [24]:
today_data.shape

(34, 6)

In [25]:
['today_'+i for i in today_data.columns]

['today_confirm',
 'today_suspect',
 'today_heal',
 'today_dead',
 'today_severe',
 'today_storeConfirm']

In [26]:
today_data.columns = ['today_'+i for i in today_data.columns] # 由于today中键名和total键名相同，因此需要修改列名称

In [27]:
today_data.head()

,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm
0,0,None,0,0,None,0
1,0,None,0,0,None,0
2,0,None,0,0,None,0
3,0,None,0,0,None,0
4,2,None,2,0,None,0


In [28]:
# 获取total中的数据
total_data = pd.DataFrame([province['total'] for province in data_province ])
total_data.columns = ['total_'+i for i in total_data.columns]
total_data.head()

,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,total_newConfirm,total_newDead,total_newHeal
0,68135,0,63623,4512,0,0,NaN,NaN,NaN
1,1641,0,1627,8,0,0,NaN,NaN,NaN
2,1276,0,1254,22,0,0,NaN,NaN,NaN
3,1269,0,1267,1,0,0,NaN,NaN,NaN
4,1205,0,1107,7,0,0,NaN,NaN,NaN


In [29]:
total_data.shape

(34, 9)

In [30]:
pd.concat([info,total_data,today_data],axis=1).head() # 将三个数据合并

,id,lastUpdateTime,name,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,total_newConfirm,total_newDead,total_newHeal,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm
0,420000,2020-07-01 00:04:02,湖北,68135,0,63623,4512,0,0,NaN,NaN,NaN,0,None,0,0,None,0
1,440000,2020-07-01 08:23:59,广东,1641,0,1627,8,0,0,NaN,NaN,NaN,0,None,0,0,None,0
2,410000,2020-07-01 00:04:01,河南,1276,0,1254,22,0,0,NaN,NaN,NaN,0,None,0,0,None,0
3,330000,2020-07-01 00:04:02,浙江,1269,0,1267,1,0,0,NaN,NaN,NaN,0,None,0,0,None,0
4,810000,2020-07-01 10:43:48,香港,1205,0,1107,7,0,0,NaN,NaN,NaN,2,None,2,0,None,0


In [32]:
def get_data(data,info_list):
    info = pd.DataFrame(data)[info_list] # 主要信息
    
    today_data = pd.DataFrame([i['today'] for i in data ])       # 提取today的数据
    today_data.columns = ['today_'+i for i in today_data.columns] # 修改列名 columns
    
    total_data = pd.DataFrame([i['total'] for i in data ])       # 提取total的数据
    total_data.columns = ['total_'+i for i in total_data.columns] # 修改列名 columns
    
    return pd.concat([info,total_data,today_data],axis=1) # info、today和total横向合并最终得到汇总的数据

In [33]:
today_province = get_data(data_province,['id','lastUpdateTime','name'])
today_province.head()

,id,lastUpdateTime,name,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,total_newConfirm,total_newDead,total_newHeal,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm
0,420000,2020-07-01 00:04:02,湖北,68135,0,63623,4512,0,0,NaN,NaN,NaN,0,None,0,0,None,0
1,440000,2020-07-01 08:23:59,广东,1641,0,1627,8,0,0,NaN,NaN,NaN,0,None,0,0,None,0
2,410000,2020-07-01 00:04:01,河南,1276,0,1254,22,0,0,NaN,NaN,NaN,0,None,0,0,None,0
3,330000,2020-07-01 00:04:02,浙江,1269,0,1267,1,0,0,NaN,NaN,NaN,0,None,0,0,None,0
4,810000,2020-07-01 10:43:48,香港,1205,0,1107,7,0,0,NaN,NaN,NaN,2,None,2,0,None,0


In [34]:
def save_data(data,name): # 定义保存数据方法
    file_name = name+'_'+time.strftime('%Y_%m_%d',time.localtime(time.time()))+'.csv'
    data.to_csv(file_name,index=None,encoding='utf_8_sig')
    print(file_name+' 保存成功！')

In [35]:
time.strftime('%Y_%m_%d',time.localtime(time.time()))

'2020_07_01'

In [36]:
save_data(today_province,'today_province')

today_province_2020_07_01.csv 保存成功！


#### 世界各国实时数据爬取

之前已经了解到在json数据`data`中的`areaTree`是列表格式，每个元素都是一个国家的实时数据，每个元素的`children`是各国家省份的数据，现在我们提取世界各国实时数据。

<img src="https://imgkr.cn-bj.ufileos.com/88e17c8a-d72a-4f55-933a-1f935ad10703.png">

In [37]:
areaTree = data['areaTree'] # 取出areaTree

In [38]:
areaTree[0] # 查看第一个国家的数据

{'today': {'confirm': 2,
  'suspect': None,
  'heal': 2,
  'dead': 0,
  'severe': None,
  'storeConfirm': None},
 'total': {'confirm': 1174,
  'suspect': 0,
  'heal': 1031,
  'dead': 50,
  'severe': 0,
  'input': 0},
 'extData': {},
 'name': '突尼斯',
 'id': '9577772',
 'lastUpdateTime': '2020-07-01 01:08:30',
 'children': []}

`areaTree`中每个键值的含义：

|键名称|数据内容|
|--|--|
|today|当日数据|
|total|累计数据|
|extData|无任何数据|
|name|国家名称|
|id |各国编号|
|lastUpdateTime|更新时间|
|children|各国下一级数据(例如省份或州)|

In [39]:
for i in range(len(areaTree)):  # 查看各国家名称和更新时间
    print(areaTree[i]['name'],areaTree[i]['lastUpdateTime'])
    if i == 5:
        break

突尼斯 2020-07-01 01:08:30
塞尔维亚 2020-07-01 00:03:26
中国 2020-07-01 15:28:52
日本 2020-06-15 07:16:58
泰国 2020-07-01 15:48:53
新加坡 2020-07-01 00:28:29


从areaTree中提取每个国家的实时数据

In [40]:
today_world = get_data(areaTree,['id','lastUpdateTime','name']) # areaTree
today_world.head()

,id,lastUpdateTime,name,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,today_input
0,9577772,2020-07-01 01:08:30,突尼斯,1174,0,1031,50,0,0.0,2.0,NaN,2.0,0.0,NaN,NaN,NaN
1,9507896,2020-07-01 00:03:26,塞尔维亚,14564,0,12662,277,0,0.0,276.0,NaN,0.0,0.0,NaN,NaN,NaN
2,0,2020-07-01 15:28:52,中国,85232,0,80071,4648,0,1918.0,5.0,NaN,17.0,0.0,NaN,-12.0,0.0
3,1,2020-06-15 07:16:58,日本,18241,0,16298,940,0,0.0,75.0,NaN,63.0,0.0,NaN,NaN,0.0
4,2,2020-07-01 15:48:53,泰国,3173,0,3059,58,0,0.0,2.0,NaN,3.0,0.0,NaN,NaN,NaN


In [41]:
save_data(today_world,'today_world')

today_world_2020_07_01.csv 保存成功！


### 历史数据爬取
#### 全国历史数据爬取

在数据`data`中，总共有四个键，其中`chinaDayList`存放着中国的历史数据，我们将其取出

In [42]:
data.keys()

dict_keys(['chinaTotal', 'chinaDayList', 'lastUpdateTime', 'overseaLastUpdateTime', 'areaTree'])

In [43]:
chinaDayList = data['chinaDayList'] # 取出 chinaDayList

chinaDayList中键值含义：

|键名称|数据内容|
|--|--|
|date|日期|
|today|当日数据|
|total|累计数据|
|lastUpdateTime|更新时间|

<img src="https://imgkr.cn-bj.ufileos.com/34a1b1a8-ca39-411b-89b7-339641b94bf8.png">

In [44]:
type(chinaDayList) # 查看chinaDayList的格式

list

In [45]:
chinaDayList[0]

{'date': '2020-01-13',
 'today': {'confirm': 0,
  'suspect': 0,
  'heal': 0,
  'dead': 0,
  'severe': None,
  'storeConfirm': 0,
  'input': 0},
 'total': {'confirm': 41,
  'suspect': 0,
  'heal': 0,
  'dead': 1,
  'severe': None,
  'input': 0,
  'storeConfirm': 0},
 'extData': None,
 'lastUpdateTime': None}

可以看到`today`、`total`中也嵌套着字典，因此直接使用定义好的方法从`chinaDayList`中提取全国历史数据

In [46]:
alltime_China = get_data(chinaDayList,['date','lastUpdateTime'])  # date lastUpdateTime
alltime_China.head()

,date,lastUpdateTime,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,total_storeConfirm,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,today_input
0,2020-01-13,None,41,0,0,1,None,0,0,0,0,0,0,None,0,0
1,2020-01-14,None,41,0,0,1,None,0,0,0,0,0,0,None,0,0
2,2020-01-15,None,41,0,5,2,None,0,0,0,0,0,0,None,0,0
3,2020-01-16,None,45,0,8,2,None,0,0,0,0,0,0,None,0,0
4,2020-01-17,None,62,0,12,2,None,0,0,0,0,0,0,None,0,0


In [47]:
save_data(alltime_China,'alltime_China')

alltime_China_2020_07_01.csv 保存成功！


#### 全国各省历史数据爬取

找到第二个数据地址https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode=420000 ，在页面对比发现是湖北省的历史数据。
<img src="https://imgkr.cn-bj.ufileos.com/7c5f2e47-8ec5-429c-b292-377c94c2e311.png">

先以其中一个省为例，先尝试获取其历史数据，其他的省可以使用同样的方法

In [49]:
url = 'https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode=420000' # 定义数据地址
r = requests.get(url, headers=headers) # 进行请求
data_json = json.loads(r.text) # 获取json数据

In [50]:
data_json.keys()

dict_keys(['reqId', 'code', 'msg', 'data', 'timestamp'])

<img src="https://imgkr.cn-bj.ufileos.com/8b21333b-ea30-4f8b-b61a-65710f43cc4a.png">

在`list`中存放着数据

In [51]:
data_json['data']['list'][0]

{'date': '2020-01-20',
 'today': {'confirm': 0,
  'suspect': 0,
  'heal': 0,
  'dead': 0,
  'severe': None,
  'storeConfirm': 0,
  'input': 0},
 'total': {'confirm': 270,
  'suspect': 0,
  'heal': 25,
  'dead': 6,
  'severe': None,
  'input': 0,
  'storeConfirm': 239},
 'extData': None,
 'lastUpdateTime': None}

可以看出一个省一天的的数据格式如上，这和之前的数据结构一样，因此可以使用之前的方法得到数据

In [52]:
data_test = get_data(data_json['data']['list'],['date'])  
data_test['name'] = '湖北省'
data_test.head()

,date,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,total_storeConfirm,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,today_input,name
0,2020-01-20,270,0,25,6,None,0,239,0,0,0,0,None,0,0,湖北省
1,2020-01-21,375,0,28,9,None,0,338,105,0,3,3,None,0,0,湖北省
2,2020-01-22,444,0,28,17,None,0,399,69,0,0,8,None,0,0,湖北省
3,2020-01-23,549,0,31,24,None,0,494,105,0,3,7,None,0,0,湖北省
4,2020-01-24,729,0,32,39,None,0,658,180,0,1,15,None,0,0,湖北省


在湖北省历史数据的地址中，我们发现参数aeraCode=420000，而这刚好和全国各省实时数据today_province中的id对应.

In [53]:
today_province[['id','name']].head() # 输出前五行

,id,name
0,420000,湖北
1,440000,广东
2,410000,河南
3,330000,浙江
4,810000,香港


<img src="https://imgkr.cn-bj.ufileos.com/ccd2789b-9ead-4da1-b224-38525d9d6337.png">

因此为了得到每个省的历史数据，我们只需要将各省的行政代码作为参数传入这个地址即可，如下所示：<br>
`https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode=各省行政代码`<br>
例如广东省历史数据的地址为：`https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode=440000`<br>
湖南省历史数据的地址为：`https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode=430000`

但是数据中并没有显示省的名称，因此需要写入每个省的名称。
<img src="https://imgkr.cn-bj.ufileos.com/5110e7f4-d65f-4769-b803-f632927b08c0.png">

以一个简单的数据为例，展示生成字典的方法

In [54]:
a = ['1','2','3','4']
b = ['q','w','e','r']

for i,j in zip(a, b):
    print(i,j)

1 q
2 w
3 e
4 r


In [55]:
{ i:j  for i,j in zip(a, b)}

{'1': 'q', '2': 'w', '3': 'e', '4': 'r'}

In [63]:
today_province[['id','name']].head()

,id,name
0,420000,湖北
1,440000,广东
2,410000,河南
3,330000,浙江
4,810000,香港


In [65]:
province_dict = { num:name for num,name in zip(today_province['id'],today_province['name']) } # zip

In [66]:
# 查看前五个内容
count = 0
for i in province_dict:
    print(i,province_dict[i])
    count += 1
    if count == 5:
        break

420000 湖北
440000 广东
410000 河南
330000 浙江
810000 香港


每一个省的列名是相同的，因此多个省的数据合并起来就可以存入一个数据中，数据合并演示的例子如下：

In [59]:
df1 = pd.DataFrame([{'a':1,'b':2,'c':3,},{'a':111,'b':222}])
df1

,a,b,c
0,1,2,3.0
1,111,222,NaN


In [60]:
df2 = pd.DataFrame([{'a':9,'b':8,'c':7,},{'a':345,'c':789}])
df2

,a,b,c
0,9,8.0,7
1,345,NaN,789


In [61]:
df1 = pd.concat([df1,df2],axis=0)
df1

,a,b,c
0,1,2.0,3.0
1,111,222.0,NaN
0,9,8.0,7.0
1,345,NaN,789.0


按照上述方法，得到每一个省的数据后，进行合并

In [67]:
start = time.time()
for province_id in province_dict: # 遍历各省编号
    
    try:
        # 按照省编号访问每个省的数据地址，并获取json数据
        url = 'https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode='+ province_id  
        r = requests.get(url, headers=headers)
        data_json = json.loads(r.text)
        
        # 提取各省数据，然后写入各省名称
        province_data = get_data(data_json['data']['list'],['date'])
        province_data['name'] = province_dict[province_id]
        
        # 合并数据
        if province_id == '420000':
            alltime_province = province_data
        else:
            alltime_province = pd.concat([alltime_province,province_data]) 
            
        print('-'*20,province_dict[province_id],'成功',
              province_data.shape,alltime_province.shape,
              ',累计耗时:',round(time.time()-start),'-'*20)
        
        # 设置延迟等待
        time.sleep(10)
        
    except:
        print('-'*20,province_dict[province_id],'wrong','-'*20)

-------------------- 湖北 成功 (164, 16) (164, 16) ,累计耗时: 0 --------------------
-------------------- 广东 成功 (162, 16) (326, 16) ,累计耗时: 11 --------------------
-------------------- 河南 成功 (162, 16) (488, 16) ,累计耗时: 21 --------------------
-------------------- 浙江 成功 (160, 16) (648, 16) ,累计耗时: 31 --------------------
-------------------- 香港 成功 (164, 16) (812, 16) ,累计耗时: 42 --------------------
-------------------- 湖南 成功 (161, 16) (973, 16) ,累计耗时: 52 --------------------
-------------------- 安徽 成功 (161, 16) (1134, 16) ,累计耗时: 62 --------------------
-------------------- 黑龙江 成功 (153, 16) (1287, 16) ,累计耗时: 73 --------------------
-------------------- 江西 成功 (162, 16) (1449, 16) ,累计耗时: 83 --------------------
-------------------- 北京 成功 (156, 16) (1605, 16) ,累计耗时: 93 --------------------
-------------------- 山东 成功 (156, 16) (1761, 16) ,累计耗时: 104 --------------------
-------------------- 上海 成功 (148, 16) (1909, 16) ,累计耗时: 114 --------------------
-------------------- 江苏 成功 (162, 16) (2071, 16) ,累计耗时: 1

In [68]:
save_data(alltime_province,'alltime_province')

alltime_province_2020_07_01.csv 保存成功！


#### 世界各国历史数据爬取

In [69]:
url_italy = 'https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode=15'  # 意大利的数据地址
r = requests.get(url_italy, headers=headers) # 进行访问
italy_json = json.loads(r.text) # 导出json数据

In [70]:
italy_json.keys()

dict_keys(['reqId', 'code', 'msg', 'data', 'timestamp'])

In [71]:
italy_json['data'].keys()

dict_keys(['list'])

结合页面观察，发现各国家的历史数据结构和各省的类似。
<img src="https://imgkr.cn-bj.ufileos.com/ee6f9d6a-b174-4242-8789-39bf02b7af5b.png">

In [72]:
italy_json['data']['list'][0] # 查看数据内容

{'date': '2020-01-31',
 'today': {'confirm': 2,
  'suspect': 0,
  'heal': 0,
  'dead': 0,
  'severe': None,
  'storeConfirm': None},
 'total': {'confirm': 2,
  'suspect': 0,
  'heal': 0,
  'dead': 0,
  'severe': 0,
  'input': 0},
 'extData': None,
 'lastUpdateTime': None}

每个国家的数据格式和之前各省的几乎一样，因此使用定义好的方法生成数据，然后把国家名称写入数据

In [73]:
data_italy = get_data(italy_json['data']['list'],['date']) # 生成数据
data_italy['name'] = '意大利' # 写入意大利国家名称
data_italy.head()

,date,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,today_input,name
0,2020-01-31,2,0,0,0,0,0,2,0.0,0,0,NaN,None,NaN,意大利
1,2020-02-01,2,0,0,0,0,0,0,0.0,0,0,NaN,None,0.0,意大利
2,2020-02-02,2,0,0,0,0,0,0,0.0,0,0,NaN,None,0.0,意大利
3,2020-02-03,2,0,0,0,0,0,0,0.0,0,0,NaN,None,0.0,意大利
4,2020-02-04,2,0,0,0,0,0,0,0.0,0,0,NaN,None,0.0,意大利


因为原始数据中没有国家名称，为了得到每个国家的名称，需要生成国家编号和国家名称的键值对，这样就可以存储国家名称，在之前的世界各国实时数据today_world中有国家的编号和名称，可以用它来生成键值对

In [74]:
today_world[['id','name']].head()

,id,name
0,9577772,突尼斯
1,9507896,塞尔维亚
2,0,中国
3,1,日本
4,2,泰国


In [77]:
country_dict = { key:value for key,value in zip(today_world['id'],today_world[ 'name'] )}   

In [78]:
# 查看前五个内容
count = 0
for i in country_dict:
    print(i,country_dict[i])
    count += 1
    if count == 5:
        break

9577772 突尼斯
9507896 塞尔维亚
0 中国
1 日本
2 泰国


通过每个国家的编号访问每个国家历史数据的地址，然后获取每一个国家的历史数据

In [79]:
start = time.time()
for country_id in country_dict: # 遍历每个国家的编号
    
    try:
        # 按照编号访问每个国家的数据地址，并获取json数据
        url = 'https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode='+country_id   
        r = requests.get(url, headers=headers)
        json_data = json.loads(r.text)
        
        # 生成每个国家的数据
        country_data = get_data(json_data['data']['list'],['date'])
        country_data['name'] = country_dict[country_id]

        # 数据叠加
        if country_id == '9577772':
            alltime_world = country_data
        else:
            alltime_world = pd.concat([alltime_world,country_data]) 
            
        print('-'*20,country_dict[country_id],'成功',country_data.shape,alltime_world.shape,
              ',累计耗时:',round(time.time()-start),'-'*20)
        
        time.sleep(10)

    except:
        print('-'*20,country_dict[country_id],'wrong','-'*20)

-------------------- 突尼斯 成功 (105, 15) (105, 15) ,累计耗时: 0 --------------------
-------------------- 塞尔维亚 成功 (110, 15) (215, 15) ,累计耗时: 11 --------------------
-------------------- 中国 成功 (146, 15) (361, 15) ,累计耗时: 21 --------------------
-------------------- 日本 成功 (135, 15) (496, 15) ,累计耗时: 31 --------------------
-------------------- 泰国 成功 (121, 15) (617, 15) ,累计耗时: 41 --------------------
-------------------- 新加坡 成功 (138, 15) (755, 15) ,累计耗时: 52 --------------------
-------------------- 韩国 成功 (161, 15) (916, 15) ,累计耗时: 62 --------------------
-------------------- 澳大利亚 成功 (120, 15) (1036, 15) ,累计耗时: 72 --------------------
-------------------- 德国 成功 (127, 15) (1163, 15) ,累计耗时: 82 --------------------
-------------------- 美国 成功 (132, 15) (1295, 15) ,累计耗时: 93 --------------------
-------------------- 马来西亚 成功 (124, 15) (1419, 15) ,累计耗时: 103 --------------------
-------------------- 越南 成功 (95, 15) (1514, 15) ,累计耗时: 113 --------------------
-------------------- 圣巴泰勒米 成功 (30, 15) (1544, 15) ,

-------------------- 法属波利尼西亚 成功 (36, 15) (8971, 15) ,累计耗时: 1051 --------------------
-------------------- 巴林 成功 (116, 15) (9087, 15) ,累计耗时: 1061 --------------------
-------------------- 莫桑比克 成功 (78, 15) (9165, 15) ,累计耗时: 1072 --------------------
-------------------- 喀麦隆 成功 (92, 15) (9257, 15) ,累计耗时: 1082 --------------------
-------------------- 乌干达 成功 (89, 15) (9346, 15) ,累计耗时: 1093 --------------------
-------------------- 厄立特里亚 成功 (50, 15) (9396, 15) ,累计耗时: 1103 --------------------
-------------------- 刚果（布） 成功 (73, 15) (9469, 15) ,累计耗时: 1114 --------------------
-------------------- 津巴布韦 成功 (77, 15) (9546, 15) ,累计耗时: 1124 --------------------
-------------------- 丹麦 成功 (121, 15) (9667, 15) ,累计耗时: 1135 --------------------
-------------------- 阿鲁巴 成功 (34, 15) (9701, 15) ,累计耗时: 1146 --------------------
-------------------- 斐济 成功 (57, 15) (9758, 15) ,累计耗时: 1157 --------------------
-------------------- 伯利兹 成功 (56, 15) (9814, 15) ,累计耗时: 1167 --------------------
-------------------

-------------------- 牙买加 成功 (75, 15) (16968, 15) ,累计耗时: 2074 --------------------
-------------------- 格恩西岛 成功 (29, 15) (16997, 15) ,累计耗时: 2084 --------------------
-------------------- 罗马尼亚 成功 (114, 15) (17111, 15) ,累计耗时: 2094 --------------------
-------------------- 阿曼 成功 (115, 15) (17226, 15) ,累计耗时: 2104 --------------------
-------------------- 列支敦士登 成功 (48, 15) (17274, 15) ,累计耗时: 2116 --------------------
-------------------- 马达加斯加 成功 (78, 15) (17352, 15) ,累计耗时: 2126 --------------------


In [80]:
save_data(alltime_world,'alltime_world')

alltime_world_2020_07_01.csv 保存成功！
